# Mining Association Rules

In this project, we will analyze [a data set about online browsing behavior](http://snap.stanford.edu/class/cs246-data/browsing.txt), and identify significant association rules among the items. Each line represents a browsing session of a customer, and each item is represented by a string of 8 characters.

We will implement the A Priori algorithm with PySpark. The goal is to find significant association rules with $s\ge 100$ and high confidence scores.

In [ ]:
# Install Spark
# https://github.com/twistedmove/CS246/blob/master/hw1/hw1.pdf
# https://github.com/wrwwctb/Stanford-CS246-2018-2019-winter/blob/master/completed/1_2_de.py
# https://github.com/twistedmove/CS246/blob/master/hw1/hw1q2/h1q2.py
!pip install pyspark


     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 36.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=aee8d7196f4a60ef3a634d0c476471ee39d17a461bffd1e6829546b73c68215e
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
# Download the browsing data
!wget http://snap.stanford.edu/class/cs246-data/browsing.txt

--2021-04-29 18:40:18--  http://snap.stanford.edu/class/cs246-data/browsing.txt
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3458517 (3.3M) [text/plain]
Saving to: ‘browsing.txt’

browsing.txt        100%[===================>]   3.30M   947KB/s    in 3.6s    

2021-04-29 18:40:22 (947 KB/s) - ‘browsing.txt’ saved [3458517/3458517]



In [ ]:
# Create a Spark context
import pyspark

sc = pyspark.SparkContext()


In [ ]:
# Load the data file as an RDD object
lines = sc.textFile('browsing.txt')
baskets = lines.map(lambda l: l.split())
N = baskets.count()
print("N:", N)#how many lines are there

N: 31101


In [ ]:
#ret.append((tuple(sorted((basket[j], basket[i]))), 1))


In [ ]:
baskets = baskets.map(lambda b: sorted(set(b)))
baskets.take(5)

[['ELE17451', 'ELE89019', 'FRO11987', 'GRO99222', 'SNA90258'],
 ['ELE17451',
  'ELE26917',
  'ELE52966',
  'ELE91550',
  'FRO12685',
  'FRO84225',
  'FRO90334',
  'GRO12298',
  'GRO99222',
  'SNA11465',
  'SNA30755',
  'SNA80192'],
 ['DAI22896', 'ELE17451', 'FRO86643', 'GRO73461', 'SNA99873'],
 ['ELE17451', 'ELE23393', 'ELE37798', 'FRO86643', 'GRO56989', 'SNA11465'],
 ['DAI54444',
  'ELE11375',
  'ELE17451',
  'ELE28573',
  'FRO78087',
  'FRO86643',
  'GRO39357',
  'SNA11465',
  'SNA69641']]

In [ ]:
def singles_helper(basket):
    ret = []
    for item in basket:
        ret.append((item, 1))
    return ret

In [ ]:
singles_support = baskets.flatMap(singles_helper)
singles_support.take(5)

[('ELE17451', 1),
 ('ELE89019', 1),
 ('FRO11987', 1),
 ('GRO99222', 1),
 ('SNA90258', 1)]

In [ ]:
singles_support = singles_support.reduceByKey(lambda x, y: x + y)
singles_support.take(5)

[('FRO11987', 104),
 ('SNA90258', 550),
 ('ELE52966', 380),
 ('ELE91550', 23),
 ('FRO84225', 74)]

In [ ]:
print(singles_support.count())
singles_support = singles_support.filter(lambda x: x[1] >= 100)
print(singles_support.count())

12592
647


In [ ]:
singles = dict(singles_support.collect())
#singles

In [ ]:
def doubles_helper(basket):
    ret = []
    for i in range(len(basket)):
        if basket[i] in singles:
            for j in range(i):
                if basket[j] in singles:
                    ret.append(((basket[j], basket[i]), 1)) # basket is sorted
    return ret

In [ ]:
doubles_support = baskets.flatMap(doubles_helper)
doubles_support.take(5)

[(('ELE17451', 'FRO11987'), 1),
 (('ELE17451', 'GRO99222'), 1),
 (('FRO11987', 'GRO99222'), 1),
 (('ELE17451', 'SNA90258'), 1),
 (('FRO11987', 'SNA90258'), 1)]

In [ ]:
doubles_support = doubles_support.reduceByKey(lambda x, y: x + y)
doubles_support.take(5)

[(('ELE17451', 'GRO99222'), 148),
 (('FRO11987', 'SNA90258'), 2),
 (('ELE17451', 'ELE26917'), 314),
 (('ELE17451', 'GRO12298'), 36),
 (('ELE26917', 'GRO12298'), 17)]

In [ ]:
print(doubles_support.count())
doubles_support = doubles_support.filter(lambda x: x[1] >= 100)
print(doubles_support.count())

149097
1334


In [ ]:
def confidence_doubles_helper(double_support):
    double, support = double_support
    support = float(support)
    u, v = double
    uv_conf = support / singles[u]
    vu_conf = support / singles[v]
    return (('%s -> %s' % (u, v), uv_conf),
            ('%s -> %s' % (v, u), vu_conf))

In [ ]:
doubles_conf = doubles_support.flatMap(confidence_doubles_helper)
doubles_conf.take(5)

[('ELE17451 -> GRO99222', 0.03819354838709677),
 ('GRO99222 -> ELE17451', 0.16335540838852097),
 ('ELE17451 -> ELE26917', 0.08103225806451612),
 ('ELE26917 -> ELE17451', 0.13699825479930192),
 ('ELE26917 -> GRO99222', 0.08376963350785341)]

In [ ]:
doubles_conf = doubles_conf.sortBy(lambda x: (-x[1], x[0]))
doubles_conf.take(5)

[('DAI93865 -> FRO40251', 1.0),
 ('GRO85051 -> FRO40251', 0.999176276771005),
 ('GRO38636 -> FRO40251', 0.9906542056074766),
 ('ELE12951 -> FRO40251', 0.9905660377358491),
 ('DAI88079 -> FRO40251', 0.9867256637168141)]

List the top 5 association rules with highest confidence scores for itemsets of size 3. steps are as following:

1. Create a list of candidate 3-item sets by merging two frequent item pairs. Two item pairs can generate a 3-item set if they have one element in common.
2. Read the data again so that the frequency of those candidate sets can be counted. This step should be done using the MapReduce model.
3. Remove those candidates who don't reach the support threshold $s=100$.
4. Compute the confidence value for the remaining sets, and output the top 5 itemsets.


In [ ]:
doubles = dict(doubles_support.collect())
doubles

In [ ]:
def triple_helper(basket):
    ret = []
    for i in range(len(basket)):
        if basket[i] in singles:
            for j in range(i):
                if basket[j] in singles:
                  for k in range(j):
                    if basket[k] in singles:
                      ret.append((tuple(sorted([basket[k], basket[j], basket[i]])), 1)) # basket is sorted
    return ret


In [ ]:
triple_support = baskets.flatMap(triple_helper)
triple_support.take(6)

[(('ELE17451', 'FRO11987', 'GRO99222'), 1),
 (('ELE17451', 'FRO11987', 'SNA90258'), 1),
 (('ELE17451', 'GRO99222', 'SNA90258'), 1),
 (('FRO11987', 'GRO99222', 'SNA90258'), 1),
 (('ELE17451', 'ELE26917', 'ELE52966'), 1),
 (('ELE17451', 'ELE26917', 'GRO12298'), 1)]

In [ ]:
triple_support = triple_support.reduceByKey(lambda x, y: x + y)
triple_support.take(5)

[(('ELE17451', 'FRO11987', 'SNA90258'), 1),
 (('FRO11987', 'GRO99222', 'SNA90258'), 1),
 (('ELE17451', 'ELE26917', 'GRO12298'), 1),
 (('ELE17451', 'ELE26917', 'GRO99222'), 32),
 (('ELE17451', 'GRO12298', 'GRO99222'), 2)]

In [ ]:
print(triple_support.count())
triple_support = triple_support.filter(lambda x: x[1] >= 100)
print(triple_support.count())

2790766
233


In [ ]:
def confidence_Triple_helper(triple_support):
    triple, support = triple_support
    support = float(support)
    x,y,z = triple
    x_conf = support / singles[x]
    y_conf = support / singles[y]
    z_conf = support / singles[z]
    xy_conf = support / doubles[(x,y)]
    xz_conf = support / doubles[(x,z)]
    yz_conf = support / doubles[(y,z)]
    return (('%s -> %s' % (x, (y,z)), x_conf),
            ('%s -> %s'% (y, (x,z)), y_conf),
            ('%s -> %s'% (z, (x,y)), z_conf),
            ('%s -> %s'% ((x,y), z), xy_conf),
            ('%s -> %s'% ((x,z), y), xz_conf),
            ('%s -> %s'% ((y,z), x), yz_conf)
            )

In [ ]:
triple_conf = triple_support.flatMap(confidence_Triple_helper)
triple_conf.take(5)

[("ELE17451 -> ('SNA59903', 'SNA72163')", 0.0327741935483871),
 ("SNA59903 -> ('ELE17451', 'SNA72163')", 0.1425364758698092),
 ("SNA72163 -> ('ELE17451', 'SNA59903')", 0.11651376146788991),
 ("('ELE17451', 'SNA59903') -> SNA72163", 0.36182336182336183),
 ("('ELE17451', 'SNA72163') -> SNA59903", 0.46691176470588236)]

In [ ]:
triple_conf_sort= triple_conf.sortBy(lambda x: (-x[1], x[0]))
triple_conf_sort.take(5)

[("('DAI23334', 'ELE92920') -> DAI62779", 1.0),
 ("('DAI31081', 'GRO85051') -> FRO40251", 1.0),
 ("('DAI55911', 'GRO85051') -> FRO40251", 1.0),
 ("('DAI62779', 'DAI88079') -> FRO40251", 1.0),
 ("('DAI75645', 'GRO85051') -> FRO40251", 1.0)]